# <strong><u> 3a: Event Finding Notebook
This notebook is designed with the purpose of identifying gesture events based on kinematic features of the motion tracking data.

In [1]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

### <strong> Import Data

In [7]:
PARTICIPANT = 'TEST_VIDEO'
df = pd.read_csv(f"../TEST_FILES/{PARTICIPANT}_processed_data.csv")
df.head()

,time_ms,right_wrist_x,right_wrist_y,right_wrist_speed,right_wrist_delta_x,right_wrist_delta_y,right_index_x,right_index_y,right_index_speed,right_index_delta_x,right_index_delta_y
0,33.357792,0.308497,0.593710,0.130699,0.000029,0.000159,0.279574,0.233531,0.135676,0.000066,0.000037
1,43.357792,0.308791,0.595300,0.100547,0.000022,0.000122,0.280235,0.233903,0.124657,0.000057,0.000031
2,53.357792,0.309007,0.596519,0.074318,0.000015,0.000090,0.280805,0.234217,0.107718,0.000049,0.000026
3,63.357792,0.309156,0.597419,0.051978,0.000009,0.000063,0.281294,0.234477,0.086891,0.000042,0.000021
4,73.357792,0.309250,0.598053,0.033493,0.000005,0.000042,0.281710,0.234685,0.064208,0.000035,0.000016


### <Strong> Gather Peaks in the Speed Curve

In [8]:
# Using scipy's find_peaks method to find peaks in the 'right_wrist_speed' column and save 
peaks, _ = find_peaks(df['right_wrist_speed'], height=(np.percentile(df['right_wrist_speed'], 70)))  # height parameter can be adjusted based on your specific needs
# Keep the time values of the peaks
peak_times = df['time_ms'][peaks]
peak_times

396      3993.357792
406      4093.357792
427      4303.357792
442      4453.357792
454      4573.357792
            ...     
8040    80433.357792
8051    80543.357792
8063    80663.357792
8082    80853.357792
8337    83403.357792
Name: time_ms, Length: 167, dtype: float64

### <strong> Find the onset/offset of Peaks 

In [9]:
# Initialize empty lists to store onset and offset points
onsets = []
offsets = []

# Define a threshold for onset and offset detection; this can be adjusted based on your needs
onset_threshold = np.percentile(df['right_wrist_speed'], 5)
offset_threshold = np.percentile(df['right_wrist_speed'], 5)

# Loop through each peak to find the corresponding onset and offset
for peak in peaks:
    # Find onset by iterating backwards from the peak until the speed drops below the threshold
    onset = peak
    while onset > 0 and df['right_wrist_speed'][onset] > onset_threshold:
        onset -= 1
    onsets.append(df['time_ms'][onset])  # Store the actual time value instead of the index
    
    # Find offset by iterating forwards from the peak until the speed drops below the threshold
    offset = peak
    while offset < len(df) - 1 and df['right_wrist_speed'][offset] > offset_threshold:
        offset += 1
    offsets.append(df['time_ms'][offset])  # Store the actual time value instead of the index

### <strong> Event Composition
Matching pairs of onset and offsets and combining to form full gesture events.

In [10]:
# Pair each onset with the next offset to form complete gestures
events = list(zip(onsets, offsets))
events_df = pd.DataFrame(events, columns=['onset', 'offset'])

# Keep only the first instance of each onset offset pair
events_df.drop_duplicates(subset=['onset', 'offset'], inplace=True)

# Calculate the duration of each gesture
events_df['duration'] = events_df['offset'] - events_df['onset']

# Reset the DataFrame's index
events_df.reset_index(drop=True, inplace=True)

# Asign gesture id to each gesture
events_df['gesture_id'] = events_df.index + 1

# If the offset of one gesture is within 50 ms of the onset of the next, combine them into one gesture
for i in range(len(events_df) - 1):
    if events_df['onset'][i + 1] - events_df['offset'][i] < 50:
        events_df['offset'][i] = events_df['offset'][i + 1]
        events_df['duration'][i] = events_df['offset'][i] - events_df['onset'][i]
        events_df['gesture_id'][i + 1] = events_df['gesture_id'][i]

# Using the events_df DataFrame, create a new column in the original dataframe and apply the gesture_id to where they belong in the original dataframe
df['gesture_id'] = np.nan

for i in range(len(events_df)):
    start = events_df['onset'][i]
    end = events_df['offset'][i]
    df['gesture_id'] = np.where((df['time_ms'] >= start) & (df['time_ms'] <= end), events_df['gesture_id'][i], df['gesture_id'])

C:\Users\cosmo\AppData\Local\Temp\ipykernel_23580\1432655212.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\cosmo\AppData\Local\Temp\ipykernel_23580\1432655212.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\cosmo\AppData\Local\Temp\ipykernel_23580\1432655212.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
# Plot the gesture_id column to see the results
import plotly.graph_objects as go

fig = go.Figure()

# Add the first trace with the gesture_id column
fig.add_trace(go.Scatter(x=df['time_ms'], y=df['gesture_id'], name='Gesture ID'))

# Add the second trace with the right_wrist_speed column
fig.add_trace(go.Scatter(x=df['time_ms'], y=df['right_wrist_speed'], name='Right Wrist Speed', yaxis='y2'))

# Set the layout with two y-axes
fig.update_layout(
       yaxis=dict(
              title='Gesture ID',
              titlefont=dict(color='blue'),
              tickfont=dict(color='blue')
       ),
       yaxis2=dict(
              title='Right Wrist Speed',
              titlefont=dict(color='red'),
              tickfont=dict(color='red'),
              overlaying='y',
              side='right'
       ),
       # change size of the figure
       width=1000,
       height=800
)

fig.show()

In [12]:
# Save the new annotated dataframe as a csv file
df.to_csv(f'../Events/{PARTICIPANT}_events_found.csv', index=False)